<a href="https://colab.research.google.com/github/BrianArradondo/AI-predictions/blob/main/Project_1_Part_5_Brian_Arradondo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Libraries**

In [44]:
# Typical Imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


# Modeling & Preprocessing import
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import set_config
set_config(display='diagram')

# Read in the Data

In [45]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/sales_predictions.csv")

In [46]:
df.head() # This line of code reveals a look at the first 5 lines of my Data

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [47]:
df.describe(include="number") # This line of code gives a descriptive look at the numbers and stats in our data initially

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [48]:
df.isna().sum() # This line of code reveals how many missing values we have in our untouched data

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

- We have missing data in two columns 

In [49]:
df.shape # This line of code tells us the general shape of our data

(8523, 12)

In [50]:
print(f"There are {df.shape[0]}, rows and {df.shape[1]} columns in our dataframe")

There are 8523, rows and 12 columns in our dataframe


In [51]:
df.duplicated().sum() # This line of code reveals there are no duplicates in our data so there is no need to drop any duplicates

0

In [52]:
# We will approach our missing values 
df["Item_Weight"].value_counts()

12.150    86
17.600    82
13.650    77
11.800    76
15.100    68
          ..
7.275      2
7.685      1
9.420      1
6.520      1
5.400      1
Name: Item_Weight, Length: 415, dtype: int64

# Format for ML and Train Test Split

In [53]:
X = df.drop(columns = "Item_Outlet_Sales") # This line of code identifies the features and separates from the target
y = df["Item_Outlet_Sales"]
X_train,X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

In [54]:
#check for unique values
df.nunique()

Item_Identifier              1559
Item_Weight                   415
Item_Fat_Content                5
Item_Visibility              7880
Item_Type                      16
Item_MRP                     5938
Outlet_Identifier              10
Outlet_Establishment_Year       9
Outlet_Size                     3
Outlet_Location_Type            3
Outlet_Type                     4
Item_Outlet_Sales            3493
dtype: int64

In [55]:
df["Outlet_Type"].value_counts() # This line of code is just to confirm there is no redundancy or need to adjust this category 

Supermarket Type1    5577
Grocery Store        1083
Supermarket Type3     935
Supermarket Type2     928
Name: Outlet_Type, dtype: int64

###Rename the unique row names to match the standards being used

In [56]:
df["Item_Fat_Content"].value_counts() # These rows need to be renamed to reflect the names utilized for those designations

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [57]:
df = df.replace({"LF":"Low Fat", "low fat":"Low Fat","reg":"Regular"}) # This line of code helps to fix inconsistencies in our categorical data 
df["Item_Fat_Content"].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

#Making Column Selectors

##Category Selectors

In [58]:
cat_selector = make_column_selector(dtype_include="object")

cat_selector(df) # This line of code reveals the columns that have categorical Data

['Item_Identifier',
 'Item_Fat_Content',
 'Item_Type',
 'Outlet_Identifier',
 'Outlet_Size',
 'Outlet_Location_Type',
 'Outlet_Type']

###Number Selectors

In [59]:
num_selector = make_column_selector(dtype_include='number')
num_selector(df)

['Item_Weight',
 'Item_Visibility',
 'Item_MRP',
 'Outlet_Establishment_Year',
 'Item_Outlet_Sales']

#Match Transformer with Columns

In [62]:
#Add One Hot encoder
ohe = OneHotEncoder(handle_unknown="ignore")
#add the Scaler
scaler = StandardScaler()

num_tuple = (scaler, num_selector)
cat_tuple = (ohe, cat_selector)

# **Preprocessing Object for Data to Prep for ML**

## Instantiate transformers

In [63]:
# Add Imputers
freq_imputer = SimpleImputer(strategy="most_frequent")
mean_imputer = SimpleImputer(strategy="mean")


##Instantiate Pipelines 

In [66]:
# Numerical Pipeline
num_pipe = make_pipeline(mean_imputer,scaler)
num_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler())])

In [67]:
#Categorical Pipeline
cat_pipe = make_pipeline(freq_imputer, ohe)
cat_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder', OneHotEncoder(handle_unknown='ignore'))])

# Instantiate the Column transformer

In [68]:
# Tuples for Column Transformer
num_tuple = (num_pipe, num_selector)
cat_tuple = (cat_pipe, cat_selector)
# ColumnTransformer
preprocessor = make_column_transformer(num_tuple,cat_tuple)
preprocessor

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f843f4786a0>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f843f9d3580>)])

#Transformer Data

In [69]:
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f843f4786a0>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f843f9d3580>)])

In [70]:
# transform train and test
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [72]:
X_train_processed.shape

(6392, 1595)

In [76]:
X_train_processed

<6392x1595 sparse matrix of type '<class 'numpy.float64'>'
	with 70312 stored elements in Compressed Sparse Row format>